In [20]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer_all = Timer()
timer_all.reset()

In [3]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

150.43926334381104

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


294.5704460144043

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (569911, 22)
casos tb_pacientes: (569911, 48)


In [6]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [7]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [8]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)
if (len(casos_confirmados_nao_tb_pacientes) > 0) or (len(tb_pacientes_nao_casos_confirmados) > 0):
    raise Exception("diferentes")

casos_confirmados_nao_tb_pacientes: (0, 22)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [9]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [10]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [11]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))
print(all(casosc['nome'].values == casost['nome'].values))
print(all(casosc['mun_resid'].values == casost['mun_resid'].values))
print(all(casosc['mun_atend'].values == casost['mun_atend'].values))

False
False
False
False
False
False


In [13]:
timer.reset()

sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
i = 0
for (ibge_cc, df_ibge_cc), (ibge_tb, df_ibge_tb) in zip(casosc.groupby('ibge7'),casost.groupby('ibge_res_pr')):
    print(i,end=' ')
    i+=1
    if ibge_cc != ibge_tb:
        raise Exception(ibge_cc,ibge_tb)

    df_ibge_cc = df_ibge_cc.sort_values('nome_hash')
    df_ibge_tb = df_ibge_tb.sort_values('nome_hash')

    duplicados_ibge_cc = df_ibge_cc.loc[ df_ibge_cc.duplicated('nome_hash', keep=False) ]
    duplicados_ibge_tb = df_ibge_tb.loc[ df_ibge_tb.duplicated('nome_hash', keep=False) ]

    sorted_casosc = sorted_casosc.append(df_ibge_cc.loc[~df_ibge_cc.index.isin(duplicados_ibge_cc.index)])
    sorted_casost = sorted_casost.append(df_ibge_tb.loc[~df_ibge_tb.index.isin(duplicados_ibge_tb.index)])

    for (nome_cc, df_nome_cc), (nome_tb, df_nome_tb) in zip(duplicados_ibge_cc.groupby('nome_hash'),duplicados_ibge_tb.groupby('nome_hash')):
        if nome_cc != nome_tb:
            raise Exception(nome_cc,nome_tb)

        sorted_casosc = sorted_casosc.append(df_nome_cc.sort_values('idade'))
        sorted_casost = sorted_casost.append(df_nome_tb.sort_values('idade'))

print(f"casos casos_confirmados: {sorted_casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")
print(timer.stop())

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [14]:
casosc = sorted_casosc
casost = sorted_casost
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))
print(all(casosc['nome'].values == casost['nome'].values))
print(all(casosc['mun_resid'].values == casost['mun_resid'].values))
print(all(casosc['mun_atend'].values == casost['mun_atend'].values))

True
True
False
True
False
False


In [16]:
casosc = casosc.rename(columns={'ibge7':'ibge_resid','nome':'paciente','dt_diag':'data_diagnostico','comunicacao':'data_comunicacao','is':'data_1o_sintomas'})

casosc['ordem'] = casosc['ordem'].apply(int)
casosc['identificacao'] = casost['identificacao'].apply(int).values.tolist()
casosc['ibge_atend'] = casost['ibge_atend_pr'].apply(int).values.tolist()
casosc['ibge_resid'] = casosc['ibge_resid'].apply(int)

casosc.loc[casosc['rs'].isnull(),'rs'] = 99
casosc['rs'] = casosc['rs'].apply(int)

casosc['recuperado'] = [ 1 if recuperado=="Recuperado" else 0 for recuperado in casost['status'] ]
casosc['obito'] = [ 1 if obito=="SIM" else 0 for obito in casosc['obito'] ]
casosc.loc[casosc['obito']==1, 'data_com_obito'] = casost.loc[casost['obito']=="SIM", 'dt_com_obito']

casosc['ativo'] = 0
casosc.loc[(casosc['recuperado']==0) & (casosc['obito']==0 ), 'ativo'] = 1
casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'] = 0

casosc = casosc[['ordem', 'identificacao', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'obito', 'data_obito', 'data_com_obito', 'recuperado', 'ativo', 'excluir']].sort_values('ordem')

In [17]:
writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,'geral',index=None)
worksheet = writer.sheets[f"geral"]
auto_fit_columns(worksheet,casosc)

writer.save()
writer.close()

NameError: name 'auto_fit_columns' is not defined

In [21]:
print(timedelta(seconds=timer_all.stop()))

1:27:27.574432
